In [1]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)
tf.config.list_physical_devices(device_type=None)

import tensorflow as tf
import tensorflow.keras
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras import models
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import numpy as np
import h5py
from keras import callbacks 
from sklearn.model_selection import KFold

In [2]:
# load data -------------------------------------------------

num = 4000 # number of profiles
nchan = 3 # number of channels

# get spatial dimension
fname = "Out1/profile_%05d.hdf5" %(1)
hf = h5py.File(fname, 'r')
spatial = hf.get('x') # HDF5 dataset "x": shape (4096,), type "<f8">
x = np.array(spatial) #change it to np array
x_size = np.size(np.array(spatial)) #4096; Dimension: (4096,)
hf.close()

data = np.empty((num, x_size, nchan+1), dtype=float)
for i in range(0, num):
    fname = "Out1/profile_%05d.hdf5" %(i+1)
    hf = h5py.File(fname, 'r')
    data[i, :, 0] = np.array(hf.get("red"))
    data[i, :, 1] = np.array(hf.get("grn"))
    data[i, :, 2] = np.array(hf.get("blu"))
    data[i, :, 3] = np.array(hf.get("tau")) #data.shape: (4000, 4096, 4) 4000 profiles, 4096 pixels, 3 channels 
    hf.close()

#np.random.seed(4); #same set of numbers will appear every time
#np.random.shuffle(data) #number will be in the same order in data

# allocate for training and testing -------------------------
trp = 0.8; # 80 percent to train, includes validation
train_size=int(trp*num)
test_size = num - train_size #20% for testing:0.2*4000=800

# spatial slicing dimensions
os = 8 # output slice size
halo = 2 # edge cells
ts = os + halo * 2 # total input slice size including halos 12
ks = int(ts / 2) # kernel size is half of total size 6
l2r = 511 # number of slices in one profile 

In [3]:
print(train_size,test_size)

3200 800


In [4]:
# spatial slicing the profiles

image=np.zeros((l2r*num,ts,3)) 
label=np.zeros((l2r*num,os)) 

for i in range(l2r):
#cut each profile into 511 slices. 
#label: tau channel of the center of each slice(reduced domain), they ignore the edge pixel.
    img= data[:, i*os:i*os+ts, 0:nchan] #input: 12 pixels for one slice, 3 channels 
    lb = data[:, i*os+halo:i*os+halo+os, nchan] #8 pixels for label,the tau channel is the label
    image[i*num:i*num+num]=img
    label[i*num:i*num+num,:]=lb 
print('whole dataset features size: ',image.shape,'whole dataset label size: ', label.shape) #511*4000=2044000

whole dataset features size:  (2044000, 12, 3) whole dataset label size:  (2044000, 8)


In [5]:
predictions = np.empty((test_size,8)) 
predictions.shape

(800, 8)

In [6]:
# CNN over domain -------------------------------------------


def evaluate_model(train_image, test_image, train_label, test_label):
    
    ltype = 'mean_squared_error'
    type2=tf.keras.metrics.RootMeanSquaredError()
    bsize = 1024
    eps = 500
    
    model_m = models.Sequential()

    model_m.add(layers.Conv1D(nchan*100, kernel_size=ks, activation='relu', input_shape=(ts, nchan))) #kernel size 6, 100 filters for each 3 channel

    model_m.add(layers.Conv1D(nchan*80, kernel_size=4, activation='relu')) #kernel size 4, 80 filters for each 3 channel
    model_m.add(layers.Dropout(0.2))

    model_m.add(layers.Conv1D(8, kernel_size=1)) 
    model_m.add(layers.Dropout(0.2))

    model_m.add(layers.Flatten())
    model_m.add(layers.Dense(os, activation="linear"))#output slice size : 8

    model_m.summary()




    model_m.compile(optimizer='adam',loss=ltype,metrics=[ltype,type2])

    #earlystopping to find the optimal number of epochs 
    earlystopping = callbacks.EarlyStopping(monitor ="val_loss",  
                                                mode ="min", patience = 25,  
                                                restore_best_weights = True,
                                               verbose=1) 

    history = model_m.fit(train_image,train_label,batch_size = bsize,epochs = eps,validation_split=0.125,
                              callbacks =[earlystopping])#validation set(10%)/trainset(80%)=0.125

    # test and predict ---------------------------------------------

    print('test result')
    results = model_m.evaluate(test_image, test_label)
    predictions=model_m.predict(test_image)
    mse = results[0]
    rmse=results[2]

    
    return model_m,rmse,mse,history,predictions

In [7]:
n_folds=5

kf = KFold(n_splits=n_folds)
kf.get_n_splits(image)
print(kf)
ratio=int(image.shape[0]/n_folds)

X_train=np.zeros((n_folds,ratio*(n_folds-1),12,3))
y_train=np.zeros((n_folds,ratio*(n_folds-1),8))
X_test=np.zeros((n_folds,ratio,12,3))
y_test=np.zeros((n_folds,ratio,8))

count=0
for train_index, test_index in kf.split(image):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train[count], X_test[count] = image[train_index], image[test_index]
    y_train[count], y_test[count] = label[train_index], label[test_index]
    count+=1

KFold(n_splits=5, random_state=None, shuffle=False)
TRAIN: [ 408800  408801  408802 ... 2043997 2043998 2043999] TEST: [     0      1      2 ... 408797 408798 408799]
TRAIN: [      0       1       2 ... 2043997 2043998 2043999] TEST: [408800 408801 408802 ... 817597 817598 817599]
TRAIN: [      0       1       2 ... 2043997 2043998 2043999] TEST: [ 817600  817601  817602 ... 1226397 1226398 1226399]
TRAIN: [      0       1       2 ... 2043997 2043998 2043999] TEST: [1226400 1226401 1226402 ... 1635197 1635198 1635199]
TRAIN: [      0       1       2 ... 1635197 1635198 1635199] TEST: [1635200 1635201 1635202 ... 2043997 2043998 2043999]


In [8]:
cv_scores = list()
cv_scores2 = list()
model_history = list()

for n in range(n_folds):

        # split data    
    train_image, test_image, train_label, test_label = X_train[n],X_test[n],y_train[n],y_test[n]
        # evaluate model
    model_m,rmse,mse,history,predictions= evaluate_model(train_image, test_image, train_label, test_label)
    print('MSE on test set in fold'+str(n+1)+' : '+str(mse))
    print('RMSE on test set in fold'+str(n+1)+' : '+str(rmse))

    cv_scores.append(mse)
    cv_scores2.append(rmse)


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 7, 300)            5700      
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 4, 240)            288240    
_________________________________________________________________
dropout (Dropout)            (None, 4, 240)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 4, 8)              1928      
_________________________________________________________________
dropout_1 (Dropout)          (None, 4, 8)              0         
_________________________________________________________________
flatten (Flatten)            (None, 32)                0         
_________________________________________________________________
dense (Dense)                (None, 8)                 2

Epoch 63/500
1398/1398 [==============================] - 10s 7ms/step - loss: 13.1756 - mean_squared_error: 13.1756 - root_mean_squared_error: 3.6297 - val_loss: 12.3571 - val_mean_squared_error: 12.3571 - val_root_mean_squared_error: 3.5153
Epoch 64/500
1398/1398 [==============================] - 10s 7ms/step - loss: 13.2923 - mean_squared_error: 13.2923 - root_mean_squared_error: 3.6455 - val_loss: 12.4247 - val_mean_squared_error: 12.4247 - val_root_mean_squared_error: 3.5249
Epoch 65/500
1398/1398 [==============================] - 11s 8ms/step - loss: 13.0986 - mean_squared_error: 13.0986 - root_mean_squared_error: 3.6192 - val_loss: 12.5593 - val_mean_squared_error: 12.5593 - val_root_mean_squared_error: 3.5439
Epoch 66/500
1398/1398 [==============================] - 10s 7ms/step - loss: 13.3219 - mean_squared_error: 13.3219 - root_mean_squared_error: 3.6492 - val_loss: 12.1905 - val_mean_squared_error: 12.1905 - val_root_mean_squared_error: 3.4915
Epoch 67/500
1398/1398 [====

Epoch 97/500
1398/1398 [==============================] - 10s 7ms/step - loss: 12.4569 - mean_squared_error: 12.4569 - root_mean_squared_error: 3.5294 - val_loss: 11.6124 - val_mean_squared_error: 11.6124 - val_root_mean_squared_error: 3.4077
Epoch 98/500
1398/1398 [==============================] - 10s 7ms/step - loss: 12.3508 - mean_squared_error: 12.3508 - root_mean_squared_error: 3.5142 - val_loss: 11.7465 - val_mean_squared_error: 11.7465 - val_root_mean_squared_error: 3.4273
Epoch 99/500
1398/1398 [==============================] - 10s 8ms/step - loss: 12.6582 - mean_squared_error: 12.6582 - root_mean_squared_error: 3.5576 - val_loss: 11.8285 - val_mean_squared_error: 11.8285 - val_root_mean_squared_error: 3.4393
Epoch 100/500
1398/1398 [==============================] - 10s 7ms/step - loss: 12.5475 - mean_squared_error: 12.5475 - root_mean_squared_error: 3.5419 - val_loss: 12.3154 - val_mean_squared_error: 12.3154 - val_root_mean_squared_error: 3.5093
Epoch 101/500
1398/1398 [==

Epoch 164/500
1398/1398 [==============================] - 10s 7ms/step - loss: 11.8266 - mean_squared_error: 11.8266 - root_mean_squared_error: 3.4389 - val_loss: 10.8974 - val_mean_squared_error: 10.8974 - val_root_mean_squared_error: 3.3011
Epoch 165/500
1398/1398 [==============================] - 11s 8ms/step - loss: 11.6351 - mean_squared_error: 11.6351 - root_mean_squared_error: 3.4109 - val_loss: 11.5141 - val_mean_squared_error: 11.5141 - val_root_mean_squared_error: 3.3932
Epoch 166/500
1398/1398 [==============================] - 11s 8ms/step - loss: 11.6076 - mean_squared_error: 11.6076 - root_mean_squared_error: 3.4069 - val_loss: 11.2008 - val_mean_squared_error: 11.2008 - val_root_mean_squared_error: 3.3468
Epoch 167/500
1398/1398 [==============================] - 10s 7ms/step - loss: 11.7864 - mean_squared_error: 11.7864 - root_mean_squared_error: 3.4330 - val_loss: 10.6591 - val_mean_squared_error: 10.6591 - val_root_mean_squared_error: 3.2648
Epoch 168/500
1398/1398 

1398/1398 [==============================] - 10s 7ms/step - loss: 15.2986 - mean_squared_error: 15.2986 - root_mean_squared_error: 3.9111 - val_loss: 16.0661 - val_mean_squared_error: 16.0661 - val_root_mean_squared_error: 4.0082
Epoch 29/500
1398/1398 [==============================] - 11s 8ms/step - loss: 15.4347 - mean_squared_error: 15.4347 - root_mean_squared_error: 3.9285 - val_loss: 14.8512 - val_mean_squared_error: 14.8512 - val_root_mean_squared_error: 3.8537
Epoch 30/500
1398/1398 [==============================] - 10s 7ms/step - loss: 15.1423 - mean_squared_error: 15.1423 - root_mean_squared_error: 3.8912 - val_loss: 15.0836 - val_mean_squared_error: 15.0836 - val_root_mean_squared_error: 3.8838
Epoch 31/500
1398/1398 [==============================] - 10s 7ms/step - loss: 15.2421 - mean_squared_error: 15.2421 - root_mean_squared_error: 3.9041 - val_loss: 15.1825 - val_mean_squared_error: 15.1825 - val_root_mean_squared_error: 3.8965
Epoch 32/500
1398/1398 [=================

...

Epoch 75/500
1398/1398 [==============================] - 11s 8ms/step - loss: 12.6281 - mean_squared_error: 12.6281 - root_mean_squared_error: 3.5534 - val_loss: 13.8586 - val_mean_squared_error: 13.8586 - val_root_mean_squared_error: 3.7227
Epoch 76/500
1398/1398 [==============================] - 11s 8ms/step - loss: 12.4540 - mean_squared_error: 12.4540 - root_mean_squared_error: 3.5289 - val_loss: 11.8144 - val_mean_squared_error: 11.8144 - val_root_mean_squared_error: 3.4372
Epoch 77/500
1398/1398 [==============================] - 11s 8ms/step - loss: 12.5254 - mean_squared_error: 12.5254 - root_mean_squared_error: 3.5390 - val_loss: 12.0593 - val_mean_squared_error: 12.0593 - val_root_mean_squared_error: 3.4727
Epoch 78/500
1398/1398 [==============================] - 11s 8ms/step - loss: 12.4301 - mean_squared_error: 12.4301 - root_mean_squared_error: 3.5255 - val_loss: 12.9483 - val_mean_squared_error: 12.9483 - val_root_mean_squared_error: 3.5984
Epoch 79/500
1398/1398 [====

Epoch 109/500
1398/1398 [==============================] - 11s 8ms/step - loss: 12.1296 - mean_squared_error: 12.1296 - root_mean_squared_error: 3.4826 - val_loss: 10.9049 - val_mean_squared_error: 10.9049 - val_root_mean_squared_error: 3.3023
Epoch 110/500
1398/1398 [==============================] - 10s 7ms/step - loss: 11.8914 - mean_squared_error: 11.8914 - root_mean_squared_error: 3.4482 - val_loss: 11.4460 - val_mean_squared_error: 11.4460 - val_root_mean_squared_error: 3.3832
Epoch 111/500
1398/1398 [==============================] - 10s 7ms/step - loss: 11.8807 - mean_squared_error: 11.8807 - root_mean_squared_error: 3.4467 - val_loss: 11.1472 - val_mean_squared_error: 11.1472 - val_root_mean_squared_error: 3.3387
Epoch 112/500
1398/1398 [==============================] - 10s 7ms/step - loss: 11.7009 - mean_squared_error: 11.7009 - root_mean_squared_error: 3.4205 - val_loss: 11.9354 - val_mean_squared_error: 11.9354 - val_root_mean_squared_error: 3.4548
Epoch 113/500
1398/1398 

Epoch 176/500
1398/1398 [==============================] - 10s 7ms/step - loss: 11.3654 - mean_squared_error: 11.3654 - root_mean_squared_error: 3.3711 - val_loss: 10.4091 - val_mean_squared_error: 10.4091 - val_root_mean_squared_error: 3.2263
Epoch 177/500
1398/1398 [==============================] - 11s 8ms/step - loss: 11.2155 - mean_squared_error: 11.2155 - root_mean_squared_error: 3.3488 - val_loss: 10.9281 - val_mean_squared_error: 10.9281 - val_root_mean_squared_error: 3.3058
Epoch 178/500
1398/1398 [==============================] - 10s 7ms/step - loss: 11.3921 - mean_squared_error: 11.3921 - root_mean_squared_error: 3.3751 - val_loss: 10.5949 - val_mean_squared_error: 10.5949 - val_root_mean_squared_error: 3.2550
Epoch 179/500
1398/1398 [==============================] - 10s 7ms/step - loss: 11.3453 - mean_squared_error: 11.3453 - root_mean_squared_error: 3.3682 - val_loss: 10.7846 - val_mean_squared_error: 10.7846 - val_root_mean_squared_error: 3.2840
Epoch 180/500
1398/1398 

Epoch 243/500
1398/1398 [==============================] - 10s 7ms/step - loss: 11.0298 - mean_squared_error: 11.0298 - root_mean_squared_error: 3.3211 - val_loss: 9.9591 - val_mean_squared_error: 9.9591 - val_root_mean_squared_error: 3.1558
Epoch 244/500
1398/1398 [==============================] - 11s 8ms/step - loss: 10.9903 - mean_squared_error: 10.9903 - root_mean_squared_error: 3.3151 - val_loss: 10.2138 - val_mean_squared_error: 10.2138 - val_root_mean_squared_error: 3.1959
Epoch 245/500
1398/1398 [==============================] - 10s 7ms/step - loss: 11.0138 - mean_squared_error: 11.0138 - root_mean_squared_error: 3.3186 - val_loss: 10.2148 - val_mean_squared_error: 10.2148 - val_root_mean_squared_error: 3.1961
Epoch 246/500
1398/1398 [==============================] - 11s 8ms/step - loss: 10.9984 - mean_squared_error: 10.9984 - root_mean_squared_error: 3.3161 - val_loss: 9.9316 - val_mean_squared_error: 9.9316 - val_root_mean_squared_error: 3.1514
Epoch 247/500
1398/1398 [===

Epoch 277/500
1398/1398 [==============================] - 11s 8ms/step - loss: 10.9167 - mean_squared_error: 10.9167 - root_mean_squared_error: 3.3040 - val_loss: 9.9920 - val_mean_squared_error: 9.9920 - val_root_mean_squared_error: 3.1610
Epoch 278/500
1398/1398 [==============================] - 10s 7ms/step - loss: 10.7263 - mean_squared_error: 10.7263 - root_mean_squared_error: 3.2750 - val_loss: 10.1560 - val_mean_squared_error: 10.1560 - val_root_mean_squared_error: 3.1869
Epoch 279/500
1398/1398 [==============================] - 11s 8ms/step - loss: 10.7352 - mean_squared_error: 10.7352 - root_mean_squared_error: 3.2764 - val_loss: 10.1547 - val_mean_squared_error: 10.1547 - val_root_mean_squared_error: 3.1866
Epoch 280/500
1398/1398 [==============================] - 11s 8ms/step - loss: 10.7950 - mean_squared_error: 10.7950 - root_mean_squared_error: 3.2854 - val_loss: 10.1754 - val_mean_squared_error: 10.1754 - val_root_mean_squared_error: 3.1899
Epoch 281/500
1398/1398 [=

In [9]:
#quantitative result: average MSE on test set and its standard deviation
print('Estimated MSE %.4f with standard deviation (%.4f)' % (np.mean(cv_scores),np.std(cv_scores)))
print('Estimated RMSE %.4f with standard deviation (%.4f)' % (np.mean(cv_scores2),np.std(cv_scores2)))

Estimated MSE 9.9901 with standard deviation (0.4772)
Estimated RMSE 3.1598 with standard deviation (0.0765)


In [14]:
cv_scores

[9.1748628616333,
 10.387792587280273,
 10.218992233276367,
 10.178900718688965,
 10.370072364807129]

In [15]:
cv_scores2

[3.029003381729126,
 3.2230098247528076,
 3.196716070175171,
 3.190438747406006,
 3.220259666442871]